In [17]:
import pandas as pd
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import requests as re
import json
import geopandas
import shapely
from striprtf.striprtf import rtf_to_text

In [18]:
link='https://data.wa.gov/resource/jjrr-uy75.json'
token='VfAaN5yV9GkIulUGGM79nWYlN'

In [19]:
schools=pd.read_csv('School_Data.csv',skiprows=11)

In [238]:
print(schools.shape)
schools.columns

(525, 22)


Index(['NCES__School_ID', 'State_School_ID', 'NCES_District_ID',
       'State_District_ID', 'Low_Grade', 'High_Grade', 'Name', 'District',
       'County', 'Address', 'City', 'State', 'ZIP', 'Area_Type', 'Charter',
       'Magnet', 'Students', 'Teachers', 'ST_Ratio', 'Free_Lunch',
       'Reduced_Lunch', 'GPS'],
      dtype='object')

In [5]:
schools.head()

,NCES School ID,State School ID,NCES District ID,State District ID,Low Grade*,High Grade*,School Name,District,County Name*,Street Address,...,Locale*,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*
0,530771001127,WA-17001-2138,5307710,WA-17001,KG,05,Adams Elementary School,Seattle School District No. 1,King County,6110 28 AV NW,...,City: Large,No,No,No,†,498.0,30.07000,16.6000000,44.00000,11.00000
1,530282000430,WA-17210-3519,5302820,WA-17210,PK,05,Adelaide Elementary School,Federal Way School District,King County,1635 SW 304th ST,...,Suburb: Large,No,No,Yes,Yes,402.0,25.01000,16.1000000,208.00000,50.00000
2,530771001249,WA-17001-3774,5307710,WA-17001,06,08,Aki Kurose Middle School,Seattle School District No. 1,King County,3928 S GRAHAM ST,...,City: Large,No,No,Yes,Yes,657.0,41.68000,15.8000000,370.00000,102.00000
3,530792001297,WA-17412-3674,5307920,WA-17412,07,08,Albert Einstein Middle School,Shoreline School District,King County,19343 3rd Avenue N.W.,...,Suburb: Large,No,No,No,†,713.0,35.39000,20.1000000,136.00000,52.00000
4,530423002062,WA-17414-4256,5304230,WA-17414,KG,05,Alcott Elementary,Lake Washington School District,King County,4213 - 228th AVE NE,...,Suburb: Large,No,No,No,†,715.0,40.13000,17.8000000,16.00000,5.00000


In [6]:
schools.rename({'County Name*':'County','Street Address':'Address',
                'School Name':'Name','NCES School ID':'NCES__School_ID',
                'State School ID':'State_School_ID','NCES District ID':'NCES_District_ID',
                'State District ID':'State_District_ID','Locale*':'Area_Type',
                'Magnet*':'Magnet','Students*':'Students','Teachers*':'Teachers',
                'Student Teacher Ratio*':'ST_Ratio','Free Lunch*':'Free_Lunch',
                'Reduced Lunch*':'Reduced_Lunch','Low Grade*':'Low_Grade',
                'High Grade*':'High_Grade'
               },
               inplace=True,
               axis=1)

In [7]:
schools=schools.drop(['Title I School*','Title 1 School Wide*','ZIP 4-digit','Phone','Locale Code*'],axis=1)

In [8]:
schools[['Address','City','State','ZIP']]

,Address,City,State,ZIP
0,6110 28 AV NW,SEATTLE,WA,98107
1,1635 SW 304th ST,Federal Way,WA,98023
2,3928 S GRAHAM ST,Seattle,WA,98118
3,19343 3rd Avenue N.W.,Shoreline,WA,98177
4,4213 - 228th AVE NE,Redmond,WA,98053
...,...,...,...,...
520,19819 136 NE,Woodinville,WA,98072
521,26454 16TH AV S,DES MOINES,WA,98198
522,12225 NE 160,Bothell,WA,98011
523,12619 SE 20th Place,Bellevue,WA,98005


In [236]:
schools.loc[1]

NCES__School_ID                           530282000430
State_School_ID                          WA-17210-3519
NCES_District_ID                               5302820
State_District_ID                             WA-17210
Low_Grade                                           PK
High_Grade                                          05
Name                        Adelaide Elementary School
District                   Federal Way School District
County                                     King County
Address                               1635 SW 304th ST
City                                       Federal Way
State                                               WA
ZIP                                              98023
Area_Type                                Suburb: Large
Charter                                             No
Magnet                                              No
Students                                           402
Teachers                                      25.01000
ST_Ratio  

In [237]:
schools.District.value_counts()

Seattle School District No. 1              110
Lake Washington School District             56
Federal Way School District                 46
Kent School District                        44
Highline School District                    41
Bellevue School District                    31
Renton School District                      29
Issaquah School District                    28
Northshore School District                  23
Auburn School District                      23
Shoreline School District                   19
Snoqualmie Valley School District           14
Enumclaw School District                    10
Tahoma School District                      10
Riverview School District                    9
Tukwila School District                      8
Mercer Island School District                6
Vashon Island School District                5
Lake Washington Institute of Technology      2
Skykomish School District                    2
Renton Technical College                     2
Rainier Valle

In [10]:
geolocater=Nominatim(user_agent='king_county_schools')
def gps_lookup(r):
    key=str(r.Name) +' '+ str(r.City)
    try:
        l=geolocater.geocode(key)
        return l.point[0:2]
    except:
        return None

In [7]:
geolocater=Nominatim(user_agent='king_county_schools')

In [14]:
l=geolocater.reverse((47.6049412,-122.3326903))

In [15]:
l

Location(Central Building, 810, 3rd Avenue, West Edge, First Hill, Seattle, King County, Washington, 98104, United States, (47.6047466, -122.33280075780988, 0.0))

In [18]:
schools.to_pickle('schools_data.pkl')

In [25]:
schools.ZIP.value_counts()

98003    19
98052    14
98042    14
98034    14
98030    14
         ..
98093     1
98039     1
98272     1
98057     1
98047     1
Name: ZIP, Length: 81, dtype: int64

In [20]:
d_parms={'schoolyear':'2014-15',
       'organizationlevel':'District',
       'gradelevel':'All Grades','testadministration':'SBAC',
       'county':'King',
       '$$app_token':token,'$limit':50000}

s_parms={'schoolyear':'2014-15',
       'organizationlevel':'School','currentschooltype':'P', #only public schools...
       'gradelevel':'All Grades','testadministration':'SBAC',
       'county':'King',
       '$$app_token':token,'$limit':50000}

In [21]:
response=re.get(link,params=d_parms)
response.status_code

200

In [22]:
district_grades=pd.read_json(response.text)
print(district_grades.shape)
district_grades.columns

(946, 27)


Index(['schoolyear', 'organizationlevel', 'county', 'esdname',
       'esdorganizationid', 'districtcode', 'districtname',
       'districtorganizationid', 'schoolname', 'studentgrouptype',
       'studentgroup', 'gradelevel', 'test_administration_group',
       'testadministration', 'testsubject', 'suppression',
       'count_of_students_expected',
       'count_of_students_expected_to_test_including_previously_passed',
       'countmetstandard', 'percentmetstandard', 'percentlevel1',
       'percentlevel2', 'percentlevel3', 'percentlevel4',
       'percentmettestedonly', 'percent_no_score', 'dataasof'],
      dtype='object')

In [24]:
district_grades.districtorganizationid.value_counts()

100174    48
100222    48
100105    48
100117    48
100127    48
100080    48
100216    48
100229    48
100016    48
100086    48
100236    46
100019    46
100263    46
100146    46
100240    46
100111    46
100243    46
100279    44
105683    36
104276    32
100238    32
Name: districtorganizationid, dtype: int64

In [212]:
len(district_grades.districtname.unique())

21

In [233]:
response=re.get(link,params=s_parms)
response.status_code

200

In [26]:
school_grades=pd.read_json(response.text)
print(school_grades.shape)
school_grades.columns

(946, 27)


Index(['schoolyear', 'organizationlevel', 'county', 'esdname',
       'esdorganizationid', 'districtcode', 'districtname',
       'districtorganizationid', 'schoolname', 'studentgrouptype',
       'studentgroup', 'gradelevel', 'test_administration_group',
       'testadministration', 'testsubject', 'suppression',
       'count_of_students_expected',
       'count_of_students_expected_to_test_including_previously_passed',
       'countmetstandard', 'percentmetstandard', 'percentlevel1',
       'percentlevel2', 'percentlevel3', 'percentlevel4',
       'percentmettestedonly', 'percent_no_score', 'dataasof'],
      dtype='object')

In [27]:
school_grades.districtname.value_counts()

Lake Washington School District                 48
Riverview School District                       48
Northshore School District                      48
Enumclaw School District                        48
Highline School District                        48
Kent School District                            48
Renton School District                          48
Seattle School District #1                      48
Federal Way School District                     48
Auburn School District                          48
Mercer Island School District                   46
Tukwila School District                         46
Bellevue School District                        46
Shoreline School District                       46
Issaquah School District                        46
Tahoma School District                          46
Snoqualmie Valley School District               46
Vashon Island School District                   44
First Place Scholars Charter School District    36
Skykomish School District      

In [222]:
school_grades.currentschooltype.value_counts()

P    18182
A     2080
S      357
R      244
I       96
5       72
J       44
Z       32
Q       30
Name: currentschooltype, dtype: int64

In [218]:
school_grades.dropna(subset=['percentmetstandard']).shape

(21137, 30)

In [192]:
school_grades[school_grades.testadministration=='SBAC'].testsubject.value_counts()

English Language Arts    10569
Math                     10568
Name: testsubject, dtype: int64

In [221]:
school_grades.loc[2]

schoolyear                                                                                             2014-15
organizationlevel                                                                                       School
county                                                                                                    King
esdname                                                           Puget Sound Educational Service District 121
esdorganizationid                                                                                       100006
districtcode                                                                                             17408
districtname                                                                            Auburn School District
districtorganizationid                                                                                  100016
schoolcode                                                                                                3825
s